# PRACTICA 08

### Integrantes:
* Piminchumo Leyva, Giusseppe Amaru
* Polo Gonzalez, Diego Jesus
* Salirrosas Castro, Anghelo Joseph

## Instrucciones:
### 1.	Revise e implemente la interfaz basandose en el snippet de la pagina siguiente.


In [2]:
import tkinter as tk
from PIL import Image, ImageTk
import cv2
import numpy as np

def process_frame():
    _, frame = cap.read()

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    lower = np.array([0, 90, 50])
    upper = np.array([60, 220, 180])

    mask = cv2.inRange(hsv, lower, upper)
    res = cv2.bitwise_and(frame, frame, mask=mask)

    update_label(frame, label_frame)
    update_label(mask, label_mask)
    update_label(res, label_result)

    ventana.after(20, process_frame)

def update_label(image, label):
    if len(image.shape) == 2:  # Si la imagen tiene un solo canal
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    else:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    image = resize_image(image, 300)
    image = Image.fromarray(image)
    image_tk = ImageTk.PhotoImage(image=image)
    label.config(image=image_tk)
    label.image = image_tk


def resize_image(image, max_width):
    original_height, original_width, _ = image.shape
    ratio = max_width / original_width
    height = int(original_height * ratio)
    return cv2.resize(image, (max_width, height))

ventana = tk.Tk()
ventana.title("Real-time Image Processing with Tkinter")

cap = cv2.VideoCapture(0)

label_frame = tk.Label(ventana)
label_frame.grid(row=0, column=0, padx=10, pady=10)

label_mask = tk.Label(ventana)
label_mask.grid(row=0, column=1, padx=10, pady=10)

label_result = tk.Label(ventana)
label_result.grid(row=0, column=2, padx=10, pady=10)

process_frame()

ventana.mainloop()

cap.release()
cv2.destroyAllWindows()

### 2.	Implemente un algoritmo que le permita realizar la operación de balance de blancos.

In [1]:
import tkinter as tk
from PIL import Image, ImageTk
import cv2
import numpy as np

class BalanceBlancosGUI:
    def __init__(self, master):
        self.master = master
        self.master.title("Balance de Blancos en Tiempo Real")

        self.cap = cv2.VideoCapture(0)

        self.label_frame = tk.Label(self.master, text="Original")
        self.label_frame.grid(row=0, column=0, padx=30, pady=30)

        self.label_mask = tk.Label(self.master, text="YUV")
        self.label_mask.grid(row=0, column=1, padx=30, pady=30)

        self.label_result = tk.Label(self.master, text="Balance de Blancos")
        self.label_result.grid(row=0, column=2, padx=30, pady=30)

        self.process_frame()

    def balanceBlancos(self, img_yuv):
        # Calcular la ganancia de color basada en estadísticas de píxeles representativos
        representative_pixels = img_yuv[::10, ::10, 1:3]  # Tomar muestras de U y V
        u_gain = 128 / np.mean(representative_pixels[:,:,0])
        v_gain = 128 / np.mean(representative_pixels[:,:,1])

        # Ajustar los canales U y V con las ganancias calculadas
        img_yuv[:,:,1] = np.clip(img_yuv[:,:,1] * u_gain, 0, 255)
        img_yuv[:,:,2] = np.clip(img_yuv[:,:,2] * v_gain, 0, 255)

        # Convertir de nuevo a BGR
        img_output = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
        return img_output   

    def process_frame(self):
        _, frame = self.cap.read()

        yuv = cv2.cvtColor(frame, cv2.COLOR_BGR2YUV)
        res = self.balanceBlancos(yuv)
        
        self.update_label(frame, self.label_frame)
        self.update_label(yuv, self.label_mask)
        self.update_label(res, self.label_result)

        self.master.after(20, self.process_frame)

    def update_label(self, image, label):
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
        image = self.resize_image(image, 300)
        image = Image.fromarray(image)
        image_tk = ImageTk.PhotoImage(image=image)

        label.config(image=image_tk)
        label.image = image_tk

        # Mostrar etiqueta de texto encima de la imagen
        text_label = tk.Label(self.master, text=label["text"], font=("Arial", 12), bg="white")
        text_label.place(x=label.winfo_x(), y=label.winfo_y() - 20)


    def resize_image(self, image, max_width):
        original_height, original_width, _ = image.shape
        ratio = max_width / original_width
        height = int(original_height * ratio)
        return cv2.resize(image, (max_width, height))

    def on_closing(self):
        self.cap.release()
        self.master.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = BalanceBlancosGUI(root)
    root.protocol("WM_DELETE_WINDOW", app.on_closing)
    root.mainloop()


### 3.	Implemente una interfaz que permita ingresar los 3 valores hsv (o cualquier otro espacio de color) y segmentarlos (Agregar 3 TextInput/sliders y un botón).

In [3]:
import cv2
import tkinter as tk
from tkinter import messagebox
import colorsys
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

class ColorPickerApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Segmentar Por Color")

        self.h_var = tk.DoubleVar()
        self.s_var = tk.DoubleVar()
        self.v_var = tk.DoubleVar()

        self.create_widgets()

    def create_widgets(self):
        # Sliders para H, S y V
        tk.Label(self.master, text="H:").grid(row=0, column=0, padx=5, pady=5)
        h_slider = tk.Scale(self.master, from_=0, to=179, variable=self.h_var, orient=tk.HORIZONTAL, command=self.update_labels)
        h_slider.grid(row=0, column=1, padx=5, pady=5)
        tk.Label(self.master, text="H: {:.0f}".format(self.h_var.get()), name="label_h").grid(row=0, column=2, padx=5, pady=5)

        tk.Label(self.master, text="S:").grid(row=1, column=0, padx=5, pady=5)
        s_slider = tk.Scale(self.master, from_=0, to=255, variable=self.s_var, orient=tk.HORIZONTAL, command=self.update_labels)
        s_slider.grid(row=1, column=1, padx=5, pady=5)
        tk.Label(self.master, text="S: {:.0f}".format(self.s_var.get()), name="label_s").grid(row=1, column=2, padx=5, pady=5)

        tk.Label(self.master, text="V:").grid(row=2, column=0, padx=5, pady=5)
        v_slider = tk.Scale(self.master, from_=0, to=255, variable=self.v_var, orient=tk.HORIZONTAL, command=self.update_labels)
        v_slider.grid(row=2, column=1, padx=5, pady=5)
        tk.Label(self.master, text="V: {:.0f}".format(self.v_var.get()), name="label_v").grid(row=2, column=2, padx=5, pady=5)

        # Botones
        tk.Button(self.master, text="Seleccionar Color", command=self.select_color).grid(row=3, column=0, columnspan=3, pady=10)
        tk.Button(self.master, text="Segmentar", command=self.segment).grid(row=4, column=0, columnspan=3, pady=10)

        # Canvas para mostrar el color seleccionado
        self.color_canvas = tk.Canvas(self.master, width=50, height=50, background="white")
        self.color_canvas.grid(row=0, column=3, rowspan=5, padx=10, pady=10)

    def update_labels(self, event=None):
        # Actualizar las etiquetas con los valores de los sliders
        h_label = "H: {:.0f}".format(self.h_var.get())
        s_label = "S: {:.0f}".format(self.s_var.get())
        v_label = "V: {:.0f}".format(self.v_var.get())

        self.master.nametowidget("label_h").config(text=h_label)
        self.master.nametowidget("label_s").config(text=s_label)
        self.master.nametowidget("label_v").config(text=v_label)

    def select_color(self):
        try:
            h = int(self.h_var.get())
            s = int(self.s_var.get())
            v = int(self.v_var.get())

            # Verificar que los valores estén en el rango correcto
            if 0 <= h <= 179 and 0 <= s <= 255 and 0 <= v <= 255:
                rgb_color = self.hsv_to_rgb(h / 179.0, s / 255.0, v / 255.0)
                hex_color = "#{:02X}{:02X}{:02X}".format(*rgb_color)

                self.color_canvas.config(background=hex_color)
            else:
                messagebox.showwarning("Advertencia", "Por favor, ingrese valores válidos.")
        except ValueError:
            messagebox.showwarning("Advertencia", "Por favor, ingrese valores numéricos.")

    def hsv_to_rgb(self, h, s, v):
        return tuple(round(i * 255) for i in colorsys.hsv_to_rgb(h, s, v))

    def segment(self):
        try:
            h = int(self.h_var.get())
            s = int(self.s_var.get())
            v = int(self.v_var.get())

            # Verificar que los valores estén en el rango correcto
            if 0 <= h <= 179 and 0 <= s <= 255 and 0 <= v <= 255:
                lower = np.array([max(0, h - 70), max(0, s - 70), max(0, v - 70)])
                upper = np.array([min(179, h + 70), min(255, s + 70), min(255, v + 70)])

                cap = cv2.VideoCapture(0)

                while True:
                    _, frame = cap.read()

                    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

                    mask = cv2.inRange(hsv, lower, upper)

                    res = cv2.bitwise_and(frame, frame, mask=mask)

                    # Mostrar imágenes
                    cv2.imshow('Original', frame)
                    cv2.imshow('Mask', mask)
                    cv2.imshow('Segmentado', res)

                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        break

                cap.release()
                cv2.destroyAllWindows()
            else:
                messagebox.showwarning("Advertencia", "Por favor, ingrese valores válidos.")
        except ValueError:
            messagebox.showwarning("Advertencia", "Por favor, ingrese valores numéricos.")

def main():
    root = tk.Tk()
    app = ColorPickerApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()
